### 1. Create Spark tables pointing to the Azure Cosmos DB Analytical Store collections using Azure Synapse Link 


In [ ]:
%%sql
create database CosmosDemoIoT

In [ ]:
%%sql

create table if not exists CosmosDemoIoT.IoTSignals
using cosmos.olap
options(spark.synapse.linkedService 'CosmosDemo',
        spark.cosmos.container 'IoTSignals')

In [ ]:
%%sql

create table if not exists CosmosDemoIoT.IoTDeviceInfo
using cosmos.olap
options(spark.synapse.linkedService 'CosmosDemo',
        spark.cosmos.container 'IoTDeviceInfo')

### 2. Perform Joins across collections, apply filters and aggregations using Spark SQL 


In [ ]:
df_RPM_details = spark.sql("select a.deviceid \
                                 , b.devicetype \
                                 , cast(b.location as string) as location\
                                 , cast(b.latitude as float) as latitude\
                                 , cast(b.longitude as float) as  longitude\
                                 , a.measuretype \
                                 , a.unitSymbol \
                                 , cast(sum(measureValue) as float) as measureValueSum \
                                 , count(*) as count \
                            from CosmosDemoIoT.IoTSignals a \
                            left join CosmosDemoIoT.IoTDeviceInfo b \
                            on a.deviceid = b.deviceid \
                            where a.unitSymbol = 'RPM' \
                            group by a.deviceid, b.devicetype, b.location, b.latitude, b.longitude, a.measuretype, a.unitSymbol")

### 3. View or chart the result-set using display() function


In [ ]:
display(df_RPM_details)

### 4. Visualizations using plotly and displayHTML()
The below shows a heatmap of IoT signals across diffrent locations


In [ ]:
from plotly.offline import plot
import plotly.express as px

df_RPM_details_pd = df_RPM_details.toPandas()
fig = px.scatter_mapbox(df_RPM_details_pd, 
                        lat='latitude', 
                        lon='longitude', 
                        size = 'measureValueSum',
                        color = 'measureValueSum',
                        hover_name = 'location',
                        hover_data = ['measureValueSum','location'],
                        size_max = 30,
                        color_continuous_scale = px.colors.carto.Temps,
                        zoom=3,
                        height=600,
                        width =900)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

p = plot(fig,output_type='div')
displayHTML(p)       